In [ ]:
#dimensionality
##加速训练；方便可视化
##维度越高，任意两点的距离越远。类似地，维度越高，过拟合风险越大。

#映射
##在现实中，样本并不是随特征均匀分布，可能一些特征是常数，另一些特征高度相关，这种情况下，
###就可以将高位特征映射到低位子空间。

#流形学习

##当原始的数据的流形不是线性的，则简单的映射并不能取得好的降维效果
##流形假设：多数高维数据可以嵌入到一个低维流形中。预测任务在低位空间流形上表示更简单。

#PCA
##第一步找到与数据最接近的超平面，然后将数据映射到超平面上。
##寻找保留最大方差（最小平方和）的映射，从而可以保留最多的信息。
##PCA方法会依次找到保留最大方差、第二、第三方法的低维平面。
##PCA主成分的方向并不固定。
##PCA的主要方法是SVD，X=U*Σ*V^T,其中V^T包含了所有的主成分。
#X_d-proj=X*W_d需要进行中心化

from sklearn.datasets import fetch_mldata
from scipy.io import loadmat

mnist_raw = loadmat('C:\\Users\\hzg0601\\datasets\\mnist-original.mat')
mnist = {
        "data": mnist_raw["data"].T,
        "target": mnist_raw["label"][0],
        "COL_NAMES": ["label", "data"],
        "DESCR": "mldata.org dataset: mnist-original",
    }

#fetch_mldata?
#mnist = fetch_mldata('MNIST original')
X,y=mnist['data'],mnist['target']

X_centered=X-X.mean(axis=0)
U,s,V=np.linalg.svd(X_centered)
X_2D=X_centered.dot(V.T[:,:2])

#按数目选取
from sklearn.decomposition import PCA
pca=PCA(n_component2=2)
x2D=pca.fit_transform(X)
pca.conponents_.T[:,0]#主成分
pca.explained_variance_ratio_

#按方差贡献率选择主成分个数
pca=PCA(n_components=0.95)
X_reduced=pca.fit_transform(X)

#也可以以cumsum按方差贡献画出Dimensions-Explained Variance图，在拐点处选择维度作为降维的标准。

#pca方法可以调用inverse_transform()将数据恢复
pca=PCA(n_components=154)
X_reduced=pca.fit_transform(X)
X_recovered=pca.inverse_transform(X_reduced)


#Incremental PCA支持在线学习，将整个数据集拆分成mini-batched输入IPCA算法
from sklearn.decomposition import IncrementalPCA
n_batches=100
inc_pca=IncrementalPCA(n_components=154)
for X_batch in np.array_split(X,n_batches):
    inc_pca.partial_fit(X_batch)
X_reduced=inc_pca.transform(X)

#调用np.memmap类进行增量PCA

import numpy as np
X_mm=np.memmap(X,dtype='float32',mode='readonly',shape=(m,n))
batch_size=m//n_batches
inc_pca=IncrementalPCA(n_components=154,batch_size=batch_size)
inc_pca.fit(X_mm)


##Randomized PCA
###随机PCA利用随机算法期望快速找到d主成分的近似。它的时间复杂度O(m*d^2)+O(d^3),而不是
###O(m*d^2)+O(d^3)
rnd_pca=PCA(n_components=154,svd_solver='randomized')
X_reduced=rnd_pca.fit_transform(X)


##Kernel PCA
##KPCA可以进行非线性降维，能够保持映射后的聚类特性，甚至能将依附于非线性流形上的数据降维展开。
from sklearn.decomposition import KernelPCA
rbf_pca=KernelPCA(n_components=2,kernel='rbf',gamma=0.04)
X_reduced=rbf_pca.fit_transform(X)


#选择合适的Kernel与调参
from sklearn.model_selection import GridSearchCV
from slearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('kpca',KernelPCA(n_components=2)),
    ('log_reg',LogisticRegression())
])

param_grid=[{
    'kpca__gamma':np.linspace(0.03,0.05,10),
    'kpaca__kernel':['rbf','sigmoid']
}]

grid_serch=GridSearchCV(clf,param_grid,cv=3)
gird_search.fit(X,y)

print(grid_search.best_params_)

##无监督方法：寻找最小重建误差的kernel和超参数
##降维后，将数据映射会特征空间，找到原始数据与映射数据的对应点，计算二者的距离，
#即pre-image误差、
#pre-image误差
rbf_pca=KernelPCA(n_components=2,kernel='rbf',gamma=0.0433,fit_inverse_transform=True)
X_reduced=rbf_pca.fit_transform(X)
X_preimage=rbf_pca.inverse_transform(X_reduced)

from sklearn.metrics import mean_squared_error as mse
mse(X,X_preimage)

##局部线性嵌入（LLE)
##LLE是一种流形学习方法，但不依赖于映射。
##LLE首先测度每个样本与近邻样本的距离，而后寻找能将近邻关系保持的低维表示。这种方法在
##噪音较少的数据中非常有效。
from sklearn.manifold import LocallyLinearEmbedding as LLE
lle=LLE(n_components=2,n_neighbors=10)
X_reduced=lle.fit_transform(X)

#1,针对每个样本，找到其最近的k个近邻样本。找到样本权重
#2，将该样本用其k个近邻样本以现行函数进行重构。对样本权重进行正则化。

##其他方法
###Multidimensional Scaling(MDS)
###Isomap
###t-Distributed Stochastic Neighbor Embedding
###Linear Discriminant Analysis